7.15 Miminum possible maximum correlation

In [1]:
import numpy as np
import cvxpy as cp
from matplotlib import pyplot as plt

In [4]:
m = 10 # number of linear functions with known variance
n = 5 # length of the random vector Z
A = np.array([
    [-1.4401 , -0.2568 , -0.4253 ,  0.3839 , -0.3007 ,  0.3794 ,  1.7744 ,  0.3712 ,  0.3996 ,  1.2323] ,
    [-0.1747 , -0.8338 ,  2.7900 ,  0.3703 ,  0.8218 ,  0.0185 , -0.7323 ,  1.0773 ,  0.9564 , -0.7745] ,
    [-0.5972 , -0.1770 ,  1.2684 ,  1.0716 ,  1.5609 , -0.9204 , -1.5823 ,  0.5888 ,  0.9646 , -0.6642] ,
    [-1.0254 ,  0.2775 ,  0.6665 , -0.2695 , -0.8905 , -1.3962 , -0.4194 ,  1.5121 , -0.5088 , -2.0358] ,
    [-1.5929 , -0.3059 ,  0.9112 , -0.5246 ,  0.2679 ,  0.0038 ,  0.7488 ,  0.8160 , -1.3199 ,  0.6035]])
sigma = np.array([ 4.6542 ,  1.2522 ,  4.5739 ,  1.4898 ,  0.9448 ,  1.5184 ,  1.9903 ,  3.8886 ,  1.5138 ,  2.4334])

In [66]:
def covariance_to_correlation(covariance_matrix):
    # Calculate the standard deviations of the variables
    std_deviations = np.sqrt(np.diag(covariance_matrix))
    
    # Calculate the correlation matrix
    correlation_matrix = covariance_matrix / np.outer(std_deviations, std_deviations)
    
    return correlation_matrix
def max_nondiagonal_element(matrix):
    rows, cols = matrix.shape
    max_nondiag = None

    for i in range(rows):
        for j in range(cols):
            if i != j:  # Exclude diagonal elements
                element = matrix[i, j]
                if max_nondiag is None or element > max_nondiag:
                    max_nondiag = element

    return max_nondiag

In [69]:
Sigma = cp.Variable((n,n), PSD=True)
l = 0 
u = 1
epsilon = 1e-3
while u - l >= epsilon:
    t = (l + u) / 2  # bisection method
    objective = cp.Minimize(0)
    constraints = [cp.vstack([cp.quad_form(a, Sigma) for a in A.T]) == cp.vstack(sigma ** 2)]
    for i in range(n-1):
        for j in range(i+1,n):
            constraints += [cp.abs(Sigma[i,j]) - t * cp.geo_mean(cp.vstack([Sigma[i,i], Sigma[j,j]])) <= 0]
    prob = cp.Problem(objective, constraints)
    prob.solve()
    if prob.status == cp.OPTIMAL or prob.status == cp.OPTIMAL_INACCURATE: 
        u = t  # minimize t
    else:
        l = t
print('rho^{max}:', np.round(t,2))
print(covariance_to_correlation(Sigma.value))
print(max_nondiagonal_element(np.abs(covariance_to_correlation(Sigma.value))))

rho^{max}: 0.62
[[ 1.          0.09183525  0.62184127  0.62186704  0.18998261]
 [ 0.09183525  1.         -0.6219448   0.62182058  0.07759998]
 [ 0.62184127 -0.6219448   1.          0.06295419  0.34451391]
 [ 0.62186704  0.62182058  0.06295419  1.          0.62157776]
 [ 0.18998261  0.07759998  0.34451391  0.62157776  1.        ]]
0.621944795403213


In [63]:
#  Disciplined Quasiconvex Programming
Sigma = cp.Variable((n,n), PSD=True)
objectives = []
for i in range(n-1):
    for j in range(i+1,n):
            objectives += [cp.abs(Sigma[i,j]) / cp.geo_mean(cp.vstack([Sigma[i,i], Sigma[j,j]]))]
objective = cp.Minimize(cp.maximum(*objectives))
constraints = [cp.vstack([cp.quad_form(a, Sigma) for a in A.T]) == cp.vstack(sigma ** 2)]
problem = cp.Problem(objective, constraints)
problem.solve(qcp=True)
assert problem.is_dqcp()

/Users/homefolder/miniconda3/lib/python3.10/site-packages/cvxpy/problems/problem.py:1385: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(
/Users/homefolder/miniconda3/lib/python3.10/site-packages/cvxpy/problems/problem.py:1385: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(
/Users/homefolder/miniconda3/lib/python3.10/site-packages/cvxpy/problems/problem.py:1385: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(
/Users/homefolder/miniconda3/lib/python3.10/site-packages/cvxpy/problems/problem.py:1385: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(
/Users/h

In [67]:
print('rho^{max}:', np.round(objective.value,2))
print(covariance_to_correlation(Sigma.value))
print(max_nondiagonal_element(np.abs(covariance_to_correlation(Sigma.value))))

rho^{max}: 0.62
[[ 1.          0.09195438  0.62171754  0.62220674  0.19250262]
 [ 0.09195438  1.         -0.62182403  0.62201738  0.07592708]
 [ 0.62171754 -0.62182403  1.          0.06288648  0.34520443]
 [ 0.62220674  0.62201738  0.06288648  1.          0.62330503]
 [ 0.19250262  0.07592708  0.34520443  0.62330503  1.        ]]
0.6233050275521628
